- Not Only SQL(NoSQL)
- 정해진 규칙이 없다. (Schema X)
- JOIN 구문 사용 불가
- key / value 이루어진 데이터를 담는다
- Database - Collection(table) - Document(row)

### db.user.find( {condition} , {column list} ) 

- db.user.find() - select * from user

- db.user.find({}, { user_id : 1, _id : 0 })- select user_id from user  # 1은 visible, 2는 invisible

- db.user.find({gender : 'f'}, {} ) - select * from user where XXXXXX

- db.user.find({user_id : 'supreme-ys', gender : 'm'}, { }) - select * from user where user_id = xxx and gender = xxx

- db.user.find( { $or :  [ {user_id : 'supreme-ys'} , {gender : 'm'} ] } , { }) - select * from user where user_id = xxx or gender = xxx

